In [1]:
using Pipe
function pz(x :: AbstractArray)
    println(typeof(x), ": ", size(x))
end
macro printval(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr," = ", $ee)))
end

In [2]:
using RecursiveAutoencoders

In [3]:
using HDF5, JLD
using Iterators


In [4]:
include("load_embeddings.jl")

load_embeddings (generic function with 1 method)

In [5]:
LL,word_indexes, indexed_words =  load_embeddings("embeddings-scaled.EMBEDDING_SIZE=50.txt");
size(LL) |> println
word_indexes |> typeof |> println
indexed_words |> typeof |> println

(50,268810)
Dict{String,Int64}
Array{String,1}


In [6]:
training_trees = load("training_sents.jld", "treebank")

5-element Array{Any,1}:
 ((("Pierre","Vinken"),(",",((("61","years"),"old"),","))),(("will",("join",(("the","board"),(("as",("a",("nonexecutive","director"))),("Nov.","29"))))),"."))                                                                                                                                                                                                                   
 (("Mr.","Vinken"),(("is",("chairman",("of",(("Elsevier","N.V."),(",",("the",("Dutch",("publishing","group")))))))),"."))                                                                                                                                                                                                                                                        
 ((("Rudolph","Agnew"),(",",(((("55","years"),"old"),("and",(("former","chairman"),("of",("Consolidated",("Gold",("Fields","PLC"))))))),","))),(("was",("named",("*-1",(("a",("nonexecutive","director")),("of",("this",("British",("industr

In [63]:
function fold(rae::RAE, tree::(Any,Any))
    function eval_child(child::String)
        eval_word_embedding(rae,child,false)
    end
    function eval_child(child::Any)
        fold(rae,child)
    end
    
    c_i = eval_child(tree[1])
    c_j = eval_child(tree[2])
    eval_merges(rae, [c_i; c_j]'')[:]
end


fold (generic function with 1 method)

In [42]:
function RecursiveAutoencoders.eval_word_embeddings(rae::RAE, tree::(Any,Any))
    function eval_child(child::String)
        eval_word_embedding(rae,child,false)
    end
    function eval_child(child::Any)
        eval_word_embeddings(rae,child)
    end
    c_i = eval_child(tree[1])
    c_j = eval_child(tree[2])
    [c_i c_j]
end

eval_word_embeddings (generic function with 3 methods)

In [39]:
function unfold(rae::RAE, tree::(String,String), pp::Embedding)
    ĉ_is, ĉ_js = reconstruct(rae, pp)
    [ĉ_is ĉ_js]
end


function unfold(rae::RAE, tree::(Any,String), pp::Embedding)
    p̂_is, ĉ_js = reconstruct(rae, pp)
    ĉ_is = unfold(rae, tree[1], p̂_is)
    [ĉ_is ĉ_js]
end

function unfold(rae::RAE, tree::(String,Any), pp::Embedding)
    ĉ_is, p̂_js = reconstruct(rae, pp)
    ĉ_js = unfold(rae, tree[2], p̂_js)
    [ĉ_is ĉ_js]
    
end

function unfold(rae::RAE, tree::(Any,Any), pp::Embedding)
    p̂_is, p̂_js = reconstruct(rae, pp)
    ĉ_is = unfold(rae, tree[1], p̂_is)
    ĉ_js = unfold(rae, tree[2], p̂_js)
    [ĉ_is ĉ_js]
end

unfold (generic function with 4 methods)

In [40]:
rae = RAE(LL,word_indexes,indexed_words);

In [67]:
#eval_scores_gradient


function UBPTS(rae::RAE, pp::Embedding, ĉ_ij::Embedding,c_ij::Embedding, tree::(String, String))

end



function δ(a::Embedding, δ_above::Vector{Float64}, W::Matrix{Float64})
    #a is the ouput of this layer: a=tanh(z) where z is the input from layer below
    #W is matrix to move to above layer, from this one
    dz = 1-a.^2 #Derivitive of a=tanh(z)
    (W'*δ_above).*dz
    
    
end

function δ(ĉ_ij::Embedding,c_ij::Embedding) 
    #Output Layer
    M = length(c_ij)#==length(ĉ_ij)
    δ_above = ĉ_ij-c_ij
    δ(ĉ_ij,δ_above, eye(M))     
end


grad_top (generic function with 2 methods)

In [76]:
tree= ("killer", "cows")
c = eval_word_embeddings(rae, tree)
c= [c[:,1], c[:,2]]
p = fold(rae,tree)
ĉ = unfold(rae,tree,p)
ĉ= [ĉ[:,1], ĉ[:,2]]
grad_top(rae, ĉ, c, tree)

100-element Array{Float64,1}:
  6.32102e-6 
 -0.0533963  
  4.46675e-8 
 -0.0449085  
  0.00564747 
 -4.94233e-5 
 -0.261981   
 -1.31843e-7 
 -9.72351e-10
 -1.55821e-5 
 -0.145269   
  1.84583e-7 
 -1.48438e-5 
  ⋮          
 -0.182976   
 -0.0120398  
  0.00139948 
 -0.00129565 
  0.0296823  
 -0.000207284
  0.000823184
  0.284631   
  3.03566e-10
  0.273958   
 -2.86376e-5 
  0.00340687 

In [75]:
c

100-element Array{Float64,1}:
  0.0708036 
  0.163581  
  0.328392  
  0.0526626 
  0.0564138 
 -0.498336  
 -0.188135  
 -0.452825  
 -0.430188  
 -0.244041  
 -0.262811  
  0.164442  
 -0.329461  
  ⋮         
 -0.133182  
  0.264526  
 -0.0278491 
 -0.13398   
  0.227585  
 -0.0487419 
  0.233536  
 -0.00790207
  0.0334298 
  0.0125151 
 -0.00868124
  0.247925  

(("Mr.","Vinken"),(("is",("chairman",("of",(("Elsevier","N.V."),(",",("the",("Dutch",("publishing","group")))))))),"."))


50x13 Array{Float64,2}:
 -0.0781008   0.0398781   -0.0990165  …  -0.316181    -0.684364   -1.29578  
 -0.0522685   0.048215    -0.207461      -0.288292     0.481716    0.104425 
 -0.097986    0.0176077    0.347496       0.00522425   0.203799    0.0333433
  0.294186   -0.0251486   -0.38533        0.0601309    0.493344   -0.123819 
  0.138973   -0.0627525    0.321955       0.474345     0.539983   -0.205738 
 -0.253114    0.0251626    0.131625   …  -0.0204928   -0.631618   -0.91893  
  0.167022    0.0117941   -0.44702       -0.150081     0.124077    0.33767  
 -0.181067    0.0613618   -0.579675      -0.430258    -0.0707779  -1.35253  
 -0.017897    0.0182828    0.0898394      0.213327     0.515492    0.0627508
 -0.174558    0.028989    -0.176701      -0.348957     0.0271393  -0.706524 
  0.272071    0.0305115   -0.101945   …   0.131612    -0.217549    0.313866 
 -0.0591885  -0.0174257    0.495167       0.260685     0.103464   -0.206052 
  0.0366249   0.0267883    0.0331284     -0.1686    

In [23]:
function cosine_dist(a,b)
    (a⋅b)/(norm(a)*norm(b))
end

function neighbour_dists(cc::Vector{Float64}, globe::Matrix{Float64})
    [cosine_dist(cc, globe[:,ii]) for ii in 1:size(globe,2)]
end


function show_best(rae::RAE,ĉ::Embedding, nbest=20)
    candidates=neighbour_dists(ĉ,rae.L)   
    best_cands = [ (findfirst(candidates,score), score)
                    for score in select(candidates,1:nbest, rev=true)[1:nbest]]
    vcat([[rae.indexed_words[ii] round(score,2)] for (ii,score) in best_cands]...)
end

function show_bests(rae::RAE,ĉs::Embeddings, nbest=20)
    hcat([show_best(rae,ĉs[:,ii],nbest) for ii in 1:size(ĉs,2)]...)
end


show_bests (generic function with 2 methods)

In [28]:
bs = show_bests(rae, ĉ_ij)
bs[1,:][1:2:end]

13-element Array{Any,1}:
 "Yasser"      
 "stand-ins"   
 "Woltjes"     
 "ephedra-free"
 "d'interprte" 
 "guerrillaa"  
 "Oregonian"   
 "machinist"   
 "Fitzpatrick" 
 "1-1/4-mile"  
 "snap-shut"   
 "five-page"   
 "swearing"    

In [29]:
ĉ_ij

50x13 Array{Float64,2}:
  0.999885   0.987141   1.0       …  -0.815472  -1.0        -0.999992
 -0.999984  -0.999373  -1.0           1.0       -0.999896   -0.999999
 -1.0        0.184681  -0.970433      1.0        1.0        -0.54884 
  0.507712   0.999675  -0.996132     -1.0       -1.0        -0.999953
  0.999995   0.999289  -1.0           0.897988   0.999969    0.949538
 -0.999999  -0.989071   1.0       …  -0.708699   0.999995   -0.997651
 -0.999996   0.685959  -0.999992      1.0       -1.0        -1.0     
 -0.928679   1.0       -0.998401     -0.998546   0.999161    0.998645
  0.565272   1.0        0.969859      1.0       -0.999847   -0.996704
 -0.999734  -1.0       -0.432486     -0.987676   0.999936    1.0     
  1.0       -0.997039  -0.996915  …   1.0       -0.99992     0.999892
  0.999979   0.430398  -0.99761      -1.0        0.999988    1.0     
 -0.999999   0.994946   1.0           0.999981  -1.0         0.19775 
  ⋮                               ⋱   ⋮                           